# Preparando o ambiente

In [1]:
from modules.utils import prepare_data_filesystem

prepare_data_filesystem()

[03/04/2024 19:11:04] Tentando criar o diretório data/raw.
[03/04/2024 19:11:04] Diretório data/raw criado com sucesso!!
[03/04/2024 19:11:04] Tentando criar o diretório data/processed.
[03/04/2024 19:11:04] Diretório data/processed criado com sucesso!!


# Programando as execuções

In [2]:
from modules.etl.first import bronze_step

bronze_step(query="dengue",languague="pt")

ModuleNotFoundError: No module named 'distutils'